In [1]:
# Append system path to include the config scripts
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from copy import deepcopy

from config import *
import lanfactory
import ssms

In [5]:
ssms.__path__

['/users/afengler/data/software/miniconda3/envs/lanfactory/lib/python3.7/site-packages/ssms']

# DATA GENERATOR CONFIGS
#### Note: Look into the ssms package documentation to get a better idea about the kinds of configs that you need for different kinds of training data


In [2]:
ssms.config.data_generator_config['lan']['mlp']

{'output_folder': 'data/lan_mlp/',
 'dgp_list': 'ddm',
 'nbins': 0,
 'n_samples': 100000,
 'n_parameter_sets': 10000,
 'n_parameter_sets_rejected': 100,
 'n_training_samples_by_parameter_set': 1000,
 'max_t': 20.0,
 'delta_t': 0.001,
 'pickleprotocol': 4,
 'n_cpus': 'all',
 'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
 'simulation_filters': {'mode': 20,
  'choice_cnt': 5,
  'mean_rt': 15,
  'std': 0,
  'mode_cnt_rel': 0.6},
 'negative_rt_cutoff': -66.77497,
 'n_subruns': 10,
 'bin_pointwise': False,
 'separate_response_channels': False}

In [3]:
#ssms.config.model_config

In [2]:
# Specify model
model = 'full_ddm'

# Where do you want to save the config file?
config_save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/'

# What kind of likelihood approximator are we generating training data for?
generator_approach = 'lan'

# Specific network type we train? (Might affect training data representation needed)
generator_network_type = 'mlp'

# Specify arguments which you want to adjust in the data generator
data_generator_arg_dict = {'dgp_list': model,
                           'n_samples': 200000,
                           'n_parameter_sets': 1000,
                           'delta_t': 0.001,
                           'n_training_samples_by_parameter_set': 2000,
                           'n_subruns': 5}

# model_config_arg_dict = {'param_bounds': [[-2.5, 0.2, 0.1, 0.0],
#                                           [2.5, 2.2, 0.9, 2.0]]}
model_config_arg_dict = {}

# Name of the config file
data_config_save_name = model + '_nsim_' + str(data_generator_arg_dict['n_samples']) + \
                        '_dt_' + str(data_generator_arg_dict['delta_t']) + \
                        '_nps_' + str(data_generator_arg_dict['n_parameter_sets']) + \
                        '_npts_' + str(data_generator_arg_dict['n_training_samples_by_parameter_set']) + '.pickle'

make_data_generator_configs(model = model,
                            generator_approach = 'lan',
                            generator_network_type = 'mlp',
                            data_generator_arg_dict = data_generator_arg_dict,
                            model_config_arg_dict = model_config_arg_dict,
                            save_name = data_config_save_name,
                            save_folder = config_save_folder)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files
Moving on...


{'model_config': {'name': 'full_ddm',
  'params': ['v', 'a', 'z', 't', 'sz', 'sv', 'st'],
  'param_bounds': [[-3.0, 0.3, 0.3, 0.25, 0.001, 0.001, 0.001],
   [3.0, 2.5, 0.7, 2.25, 0.2, 2.0, 0.25]],
  'boundary': <function ssms.basic_simulators.boundary_functions.constant(t=0)>,
  'n_params': 7,
  'default_params': [0.0, 1.0, 0.5, 0.25, 0.001, 0.001, 0.001],
  'hddm_include': ['z', 'st', 'sv', 'sz'],
  'nchoices': 2},
 'data_config': {'output_folder': 'data/lan_mlp/',
  'dgp_list': 'full_ddm',
  'nbins': 0,
  'n_samples': 200000,
  'n_parameter_sets': 1000,
  'n_parameter_sets_rejected': 100,
  'n_training_samples_by_parameter_set': 2000,
  'max_t': 20.0,
  'delta_t': 0.001,
  'pickleprotocol': 4,
  'n_cpus': 'all',
  'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
  'simulation_filters': {'mode': 20,
   'choice_cnt': 5,
   'mean_rt': 15,
   'std': 0,
   'mode_cnt_rel': 0.6},
  'negative_rt_cutoff': -66.77497,
  'n_subruns': 5,
  'bin_pointwise': False,
  'separate_response_channels':

In [3]:
x = pickle.load(open('/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/ddm_par2_no_bias_nsim_200000_dt_0.001_nps_1000_npts_2000.pickle', 'rb'))

# NETWORK AND TRAIN CONFIGS

In [8]:
# SPECIFY THE FOLLOWING VARIABLES:
# ----------------------------------------------------------------------------------------------
# Provide a unique identifier for the particular files you need from the training_data_folder
training_file_identifier = 'race_no_bias_4'
#train_file_excluder = ['par2', 'seq2', 'mic2', 'lca', 'race'] # to make sure that we load data only for a specific model (might end up using one folder per model in the end ..)
train_file_excluder = []

# Specify training data folder:
training_data_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000'

# Where do you want to save config files?
network_train_config_save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/'

# Name of the config file
hardware = 'gpu'
network_train_config_save_name = hardware + '_network_train_config_' + training_file_identifier + '_nsim_200000_dt_0.001_nps_500_npts_2000.pickle'

# Get list of training files
train_val_split = 0.98

if hardware == 'gpu':
    print('config for gpu --> use large batch size')
    batch_size = 100000
    
if hardware == 'cpu':
    print('config for cpu --> use small batch size')
    batch_size = 10000
# ----------------------------------------------------------------------------------------------

# List all files in training folder
file_list = os.listdir(training_data_folder)

# Get the files we need for our training config from the big training data folder
cnt = 0
valid_file_list = []
for file_ in file_list:
    exclude_cur = 0
    
    # Check if exclusion criterion in file name
    for excluder in train_file_excluder:
        if excluder in file_:
            exclude_cur = 1
            break
    
    # If not already rejected --> check if our identifier is in the file name
    if not exclude_cur:
        # If yes add it to valid training files for the config
        if training_file_identifier in file_:
            valid_file_list.append(training_data_folder + '/' + file_)        

val_idx_cutoff = int(train_val_split * len(valid_file_list))

# SPECIFY THE FOLLOWING VARIABLES --------------------------------------------------------------------
# Check lanfactory.config.network_config_mlp
#.      lanfactor.config.train_config_mlp for details

network_arg_dict = {'layer_types': ['dense' for i in range(4)],
                    'layer_sizes': [100, 100, 100, 1],
                    'activations': ['tanh', 'tanh', 'tanh', 'linear'],
                    'loss': 'huber',
                    'model_id': training_file_identifier
                   }
                    
train_arg_dict = {'batch_size': batch_size,
                  'n_epochs': 100,
                  'training_files': valid_file_list[:val_idx_cutoff],
                  'validation_files': valid_file_list[val_idx_cutoff:],
                  'shuffle_files': True,
                  'label_prelog_cutoff_low': 1e-7,
                  'label_prelog_cutoff_high': None,
                  'save_history': True,
                  'callbacks': ['checkpoint', 'earlystopping', 'reducelr']
                 }
# ----------------------------------------------------------------------------------------------

make_train_network_configs(save_folder = network_train_config_save_folder,
                           network_arg_dict = network_arg_dict,
                           train_arg_dict = train_arg_dict,
                           save_name = network_train_config_save_name)

config for gpu --> use large batch size
Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files
Moving on...


{'network_config': {'layer_types': ['dense', 'dense', 'dense', 'dense'],
  'layer_sizes': [100, 100, 100, 1],
  'activations': ['tanh', 'tanh', 'tanh', 'linear'],
  'loss': 'huber',
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'model_id': 'ddm'},
 'train_config': {'batch_size': 100000,
  'n_epochs': 100,
  'optimizer': 'adam',
  'learning_rate': 0.002,
  'loss': 'huber',
  'metrics': [<tensorflow.python.keras.losses.MeanSquaredError at 0x7f9b93766450>,
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'training_files': ['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_ddm_0513b2ee12aa11ec8574ac1f6b1b7afc.pickle',
   '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_ddm_b1e169f612e311ec8fd1ac1f6b63e82e.pickle',
   '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_ddm_97fbe4f

In [13]:
# Provide a unique identifier for the particular files you need from the training_data_folder
training_file_identifier = 'ddm_mic2_weibull_no_bias'

# Name of the config file
hardware = 'gpu'
dl_backend = 'torch'
network_train_config_save_name = hardware + '_' + dl_backend + '_network_train_config_' + training_file_identifier + '_nsim_200000_dt_0.001_nps_500_npts_2000_architecture_search.pickle'


# Where do you want to save config files?
network_train_config_save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/'

# Specify training data folder:
training_data_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000'

# Get list of relevant training files
file_list = os.listdir(training_data_folder)
valid_file_list = np.array([training_data_folder + '/' + \
                     file_ for file_ in file_list if training_file_identifier in file_])

# Training config hyperparameters
layer_sizes = [[100, 100, 100, 1], [100, 100, 100, 100, 1], [100, 100, 100, 100, 100, 1],
               [120, 120, 120, 1], [120, 120, 120, 120, 1], [120, 120, 120, 120, 120, 1],
               [150, 150, 150, 1], [150, 150, 150, 150, 1], [150, 150, 150, 150, 150, 1]]
layer_types = [['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
               ['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
               ['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
              ]
activations = [['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
               ['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
               ['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
              ]

train_val_split = [0.98, 0.98, 0.98, 
                   0.98, 0.98, 0.98,
                   0.98, 0.98, 0.98]

batch_size = 50000
n_epochs = 20
n_training_files = [len(valid_file_list)]

print(n_training_files)

# Loop objects
config_dict = {}
network_arg_dicts = {}
train_arg_dicts = {}    
cnt = 0

for i in range(len(layer_sizes)):
    for j in range(len(n_training_files)):
        val_idx_cutoff = int(train_val_split[i] * n_training_files[j])

        # Specify the arguments which you want to adjust in the network and train configs
        # Check lanfactory.config.network_config_mlp
        #.      lanfactor.config.train_config_mlp for details

        network_arg_dict = {'layer_types': layer_types[i],
                            'layer_sizes': layer_sizes[i],
                            'activations': activations[i],
                            'loss': ['huber'],
                            'model_id': training_file_identifier
                            }

        train_arg_dict = {'batch_size': batch_size,
                          'n_epochs': n_epochs,
                          'training_files': valid_file_list[:val_idx_cutoff],
                          'validation_files': valid_file_list[val_idx_cutoff:n_training_files[j]],
                          'shuffle_files': True,
                          'label_prelog_cutoff_low': 1e-7,
                          'label_prelog_cutoff_high': None,
                          'save_history': True,
                          'callbacks': ['checkpoint', 'earlystopping', 'reducelr']  # ['checkpoint', 'earlystopping', 'reducelr']
                          }

        config_dict[cnt] = make_train_network_configs(save_folder = network_train_config_save_folder,
                                                      network_arg_dict = network_arg_dict,
                                                      train_arg_dict = train_arg_dict,
                                                      save_name = None)
        
        print('NEW PRINT')
        print(cnt)
        #print(config_dict[0]['network_config'])
        cnt += 1

print('Now saving')
pickle.dump(config_dict, open(network_train_config_save_folder + network_train_config_save_name, 'wb'))

[500]
NEW PRINT
0
NEW PRINT
1
NEW PRINT
2
NEW PRINT
3
NEW PRINT
4
NEW PRINT
5
NEW PRINT
6
NEW PRINT
7
NEW PRINT
8
Now saving


In [12]:
config_dict[0]

{'network_config': {'layer_types': ['dense', 'dense', 'dense', 'dense'],
  'layer_sizes': [100, 100, 100, 1],
  'activations': ['tanh', 'tanh', 'tanh', 'linear'],
  'loss': ['huber'],
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'model_id': 'ddm_mic2_angle_no_bias'},
 'train_config': {'batch_size': 50000,
  'n_epochs': 20,
  'optimizer': 'adam',
  'learning_rate': 0.002,
  'loss': 'huber',
  'metrics': [<tensorflow.python.keras.losses.MeanSquaredError at 0x7f76cb286410>,
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'training_files': array(['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_ddm_mic2_angle_no_bias_e21f07d21c3d11ecb2bfac1f6b63e6e2.pickle',
         '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_ddm_mic2_angle_no_bias_50e20d521bf511ecbe06ac1f6bb2fa7c.pickle',
         '/users/afengler/data/proj_lan_pipeline/LAN_scri

In [13]:
x = pickle.load(open(network_train_config_save_folder + network_train_config_save_name, 'rb'))

In [14]:
x[0]

{'network_config': {'layer_types': ['dense', 'dense', 'dense'],
  'layer_sizes': [100, 100, 1],
  'activations': ['tanh', 'tanh', 'linear'],
  'loss': ['huber'],
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'model_id': 'race_no_bias_4'},
 'train_config': {'batch_size': 50000,
  'n_epochs': 200,
  'optimizer': 'adam',
  'learning_rate': 0.002,
  'loss': 'huber',
  'metrics': [<tensorflow.python.keras.losses.MeanSquaredError at 0x7fafc1ae55d0>,
  'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
  'training_files': array(['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_race_no_bias_4_9353d924fc6311eb95df0cc47afdbf59.pickle',
         '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_race_no_bias_4_3843ab7afbcf11eba97d0cc47afdbfb9.pickle',
         '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_20

In [12]:
for i in range(90):
    print(x[i]['train_config']['training_files'].shape[0])

50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463
50
100
200
300
463


In [3]:
x = pickle.load(open('/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/gpu_torch_network_train_config_ddm_mic2_adj_weibull_no_bias_nsim_200000_dt_0.001_nps_500_npts_2000_architecture_search.pickle', 'rb'))


In [4]:
x

{0: {'network_config': {'layer_types': ['dense', 'dense', 'dense', 'dense'],
   'layer_sizes': [100, 100, 100, 1],
   'activations': ['tanh', 'tanh', 'tanh', 'linear'],
   'loss': ['huber'],
   'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
   'model_id': 'ddm_mic2_adj_weibull_no_bias'},
  'train_config': {'batch_size': 50000,
   'n_epochs': 20,
   'optimizer': 'adam',
   'learning_rate': 0.002,
   'loss': 'huber',
   'metrics': [<tensorflow.python.keras.losses.MeanSquaredError at 0x7f76cb529e50>,
   'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
   'training_files': array([], dtype=float64),
   'validation_files': array([], dtype=float64),
   'shuffle_files': True,
   'label_prelog_cutoff_low': 1e-07,
   'label_prelog_cutoff_high': None,
   'save_history': True}},
 1: {'network_config': {'layer_types': ['dense',
    'dense',
    'dense',
    'dense',
    'dense'],
   'layer_sizes': [100, 100, 100, 100, 1],
   'activations': ['tanh', 'tanh', 'tanh', 'tanh', 'linea